In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [33]:
# Programa que permite cargar una imagen y mostrarla en pantalla con los cambios de color basados en el tipo de Daltonismo del usuario.

def cerrar_ventana(nueva_ventana):
    # Cerrar la ventana actual y mostrar la ventana principal
    nueva_ventana.destroy()
    ventana.deiconify()
def guardar_imagen(imagen_guardar):
    cv.imwrite("imagen_editada.png",imagen_guardar)
def cargar_imagen():
    global imagen_arreglo
    ruta_imagen = filedialog.askopenfilename(title="Selecciona una imagen", filetypes=[("Archivos de imagen", "*.png *.jpg *.jpeg *.gif *.bmp")])
    if ruta_imagen:
        # Abre la imagen seleccionada con Pillow (PIL)
        imagen = Image.open(ruta_imagen)
        imagen_arreglo = np.array(imagen)

        # Define el tamaño máximo en píxeles (cambiar según tus necesidades)
        ancho_maximo = 500
        alto_maximo = 300

        # Redimensiona la imagen si es necesario
        if imagen.width > ancho_maximo or imagen.height > alto_maximo:
            imagen.thumbnail((ancho_maximo, alto_maximo))

        # Convierte la imagen en un objeto Tkinter PhotoImage
        imagen_tk = ImageTk.PhotoImage(imagen)

        # Muestra la imagen en una etiqueta
        etiqueta_imagen.config(image=imagen_tk)
        etiqueta_imagen.image = imagen_tk  # ¡Importante! Mantener una referencia para evitar que se borre la imagen

        # Habilita el botón de "Umbrealización"
        boton_umbrealizacion.config(state="normal")
        # Habilita el botón de "Escala de grises"
        boton_Escala_grises.config(state="normal")
        # Habilita el botón de "Negativo"
        boton_negativo.config(state="normal")
        # Habilita el botón de "Logaritmo"
        boton_logaritmo.config(state="normal")
        # Habilita el botón de "contraste"
        boton_contraste.config(state="normal")
        # Habilita el botón de "histograma"
        boton_desenfoque.config(state="normal")

ventana = tk.Tk()
ventana.title("Ventana Principal")
ventana.configure(bg="#FFFFFF")
ventana.geometry("800x600")

titulo = tk.Label(ventana, text="ASISTENCIA PARA DALTONISMO", font=("Lucida Sans Unicode", 30), bg="#FFFFFF", fg="black")
titulo.pack()

etiqueta_imagen = tk.Label(ventana)
etiqueta_imagen.pack()

# Crea un botón de "Cargar Imagen"
boton_cargar_imagen = tk.Button(ventana, text="Cargar Imagen", font=("Lucida Sans Unicode", 15), bg="#FFFFFF", fg="gray", command=cargar_imagen, width=60, height=10)
boton_cargar_imagen.pack()

# Crea un marco para los botones de procesamiento de imagen
marco_botones = tk.Frame(ventana, bg="#FFFFFF")
marco_botones.pack(pady=60)


# Crea el botón de "Logaritmo" inicialmente deshabilitado
boton_logaritmo = tk.Button(marco_botones, text="Mostrar Imagen", font=("Lucida Sans Unicode", 15), bg="#FFFFFF", fg="gray", command=lambda: ImagenModificada(imagen_arreglo), state="disabled", width=15, height=2)
boton_logaritmo.pack(side="left", padx=10)


ventana.mainloop()